In [ ]:
import os
import mne
import shutil
import numpy as np
src_to = mne.read_source_spaces(r"D:\REFTEP_ALL\REFTEP_reco\Aalto_recon_all\fsaverage\fsaverage-oct-6-src.fif") #fsaverage src
aalto_subjects_dir = r"D:\REFTEP_ALL\REFTEP_reco\Aalto_recon_all"
potential_parcs = ['aparc','custom']
recon_all_dir = r"D:\REFTEP_ALL\REFTEP_reco"
path_in_server = r" "
pos_names = ['n15','p30','n45','p60','handknob']
for mri_dir in os.listdir(recon_all_dir):
    if 'fatsuppressed' not in mri_dir:
        subjects_mri_directory = os.path.join(recon_all_dir,mri_dir)
        site = mri_dir.split("_")[0]
        source_site = rf"D:\REFTEP_ALL\Source_analysis\Source_analysis_{site}"
        source_site_server = os.path.join(path_in_server,site)
        for subject in os.listdir(subjects_mri_directory):
            if subject != 'fsaverage' and subject != 'morph-maps':
                reftep_subject = subject.split("_reco")[0]
                subject_reftep_now = [name for name in os.listdir(source_site) if reftep_subject in name]
                if len(subject_reftep_now) not in [0,1]:
                    print("Too many identifiers for this subject")
                    break
                elif len(subject_reftep_now) == 0:
                    print(f'did not find subject {reftep_subject} in {source_site}')
                else:
                    reftep_subject = subject_reftep_now[0]
                    subject_path_server = os.path.join(source_site_server,reftep_subject)
                    sourcepath_subject = os.path.join(source_site,reftep_subject)
                    subject_mri_dir = os.path.join(subjects_mri_directory,subject)#the subject mri directory in subjects_mri_directory
                    fname_src = os.path.join(subjects_mri_directory,subject,f'{subject}-oct-6-src.fif')
                    src = mne.read_source_spaces(fname_src) #read source spaces
                    stcs_files = [file.split(".stc")[0][:-3] for file in os.listdir(subject_path_server) if "-rh" in file] #do not take -hemi.stc to the filename
                    #compute source morph
                    morph_filepath = os.path.join(subject_mri_dir,f'{subject}_fsaverage_morph.h5')
                    morph = mne.compute_source_morph(src,subject_from=subject,subject_to='fsaverage',src_to=src_to, subjects_dir=subjects_mri_directory)
                    morph.save(morph_filepath, overwrite=True)
                    #morph the source estimates
                    stcs_fsaverage = []
                    stcs_inds = []
                    for stc_file in stcs_files:
                        epoch_index = stc_file.split("_")[-1].split("-")[0]
                        stc_filepath = os.path.join(subject_path_server,stc_file)
                        stc =  mne.read_source_estimate(stc_filepath)
                        stcs_fsaverage.append(morph.apply(stc))
                        stcs_inds.append(epoch_index)
                    #load the parcellation
                    for potential_parc in potential_parcs:
                        if potential_parc == 'aparc':
                            parc_name = 'aparc'
                            labels_all = mne.read_labels_from_annot("fsaverage", "aparc", "both", subjects_dir=aalto_subjects_dir)
                            labels = [label for label in labels_all if "unknown" not in label.name]
                        elif potential_parc == 'custom':
                            #load the custom labels
                            labels = []
                            parc_name = str(pos_names)
                            for custom_label_type in pos_names:
                                custom_label_file = f"{reftep_subject}_{custom_label_type}_label_fsaverage"
                                labels_all_now = mne.read_labels_from_annot("fsaverage", custom_label_file, "both", subjects_dir=aalto_subjects_dir)
                                labels_now = [label for label in labels_all_now if "unknown" not in label.name]
                                labels = labels + labels_now
                        else:
                            raise ValueError("not valid parcellation name")
                        print(labels, len(labels))
                        label_tcs_epochs = mne.extract_label_time_course(stcs_fsaverage, labels, src_to, mode='mean_flip')
                        #check if path exists and re-create it if it does
                        subject_fsaverage_path = os.path.join(sourcepath_subject,f'{reftep_subject}_stcs_in_fsaverage_{parc_name}_depth0.8')
                        if os.path.exists(subject_fsaverage_path):
                            print(f"deleting {subject_fsaverage_path}")
                            shutil.rmtree(subject_fsaverage_path)
                        os.mkdir(subject_fsaverage_path)
                        #go through the epochs and save
                        for ind, label_tcs in zip(stcs_inds,label_tcs_epochs):
                            labelled_stc_filename = os.path.join(subject_fsaverage_path,f'{reftep_subject}-fsaverage_{parc_name}_epoch_{ind}')
                            np.save(labelled_stc_filename, label_tcs)